In [1]:
include("CGS_POMDP.jl");
pomdp = CGSPOMDP();

### Test: Points close are predicted with more accuracy and less uncertainty than points far.

In [3]:
gtlayer = pomdp.state.earth[1].gt

# seismic_line = (x1=0.0, y1=0.0, x2=5.0, y2=98.0)
well1 = Point(19, 34)
well2 = Point(30, 12)
well_close = Point(31, 14)
well_far = Point(98, 97)

all_wells = Multi([well1, well2])
data_at_all_wells = gtlayer[all_wells, :]

γ = SphericalVariogram(range=RANGE, sill=SILL, nugget=NUGGET)

okrig = GeoStatsModels.OrdinaryKriging(γ)
fitkrig = GeoStatsModels.fit(okrig, data_at_all_wells)

# We notice that prediction at close has much less variance than prediction at far, 
# and as an additional sanity check the prediction at close is close to the well2 point.
# Krigging predictprob only does one attribute at a time.

probs_far = GeoStatsModels.predictprob(fitkrig, :z, well_far)
probs_close = GeoStatsModels.predictprob(fitkrig, :z, well_close)
probs_close, probs_far

(Normal{Float64}(μ=-0.5187329251146544, σ=1.316834057971783), Normal{Float64}(μ=-1.0984231209315294, σ=2.1213203435596424))

### Test: Uncertainty is low around known points.
Additionally, as nugget is increased uncertainty (both globally and at known points) increases.
Pay attention to colorbar when verifying this.

In [20]:
function visualize_uncertainty(all_locs::Multi, layer::Int, col_name::String)
    gtlayer = pomdp.state.earth[layer].gt
    data_at_all_wells = gtlayer[all_locs, :]

    γ = SphericalVariogram(range=RANGE, sill=SILL, nugget=NUGGET) # Each feature can have a different nugget in the future.
    okrig = GeoStatsModels.OrdinaryKriging(γ)
    fitkrig = GeoStatsModels.fit(okrig, data_at_all_wells)

    df = copy(pomdp.state.earth[1].df)
    dom = domain(pomdp.state.earth[1].gt)

    var_column = Symbol("$(col_name)_variances")
    column = Symbol("$col_name")
    df[!, var_column] = [GeoStatsModels.predictprob(fitkrig, column, pt).σ
                    for pt in domain(pomdp.state.earth[1].gt)]
    var_map = georef(df, dom)
    viewer(var_map)
end

# visualize_uncertainty(all_wells, 2, "z")

visualize_uncertainty (generic function with 1 method)

### Buy well data at 3-4 locations and visualize belief

In [ ]:
buy_well_data(pomdp, 2)
buy_well_data(pomdp, 5)
buy_well_data(pomdp, 7)

In [21]:
visualize_uncertainty(Multi([pomdp.collected_locs...]), 3, "topSealThickness") # Note this is time consuming at just 3 gathered points.